# Export SalesLT Tables to Retail Data Model Bronze Layer

This notebook dynamically discovers and exports all tables from the SalesLT schema in Gaiye-SQL-DB to the retail data model bronze layer.

**Prerequisites:**
- Fabric workspace with access to Gaiye-SQL-DB
- Retail data model lakehouse attached to this notebook
- Appropriate permissions for SQL database and lakehouse access

In [ ]:
# Import required libraries (Fabric-compatible only)
import pandas as pd
from datetime import datetime
import os
import logging
import json

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Libraries imported successfully")
print(f"📅 Export started at: {datetime.now()}")
print("🔧 Using Fabric-native connectivity (no external dependencies)")

# Check if we're running in Fabric environment
try:
    spark_version = spark.version
    print(f"🏭 Fabric Spark session detected: {spark_version}")
except NameError:
    print("⚠️  Spark session not available - ensure you're running in Fabric")

In [ ]:
# Fabric Environment Diagnostic Check
print("🔍 FABRIC ENVIRONMENT DIAGNOSTIC")
print("=" * 50)

# Check 1: Spark session
try:
    print(f"✅ Spark session active: {spark.version}")
except:
    print("❌ Spark session not available")

# Check 2: Available databases
try:
    databases = spark.sql("SHOW DATABASES").toPandas()
    print(f"✅ Available databases ({len(databases)}):")
    for db in databases['databaseName']:
        print(f"   📁 {db}")
except Exception as e:
    print(f"❌ Cannot list databases: {str(e)}")

# Check 3: Lakehouse attachment
try:
    tables = spark.sql("SHOW TABLES").toPandas()
    print(f"✅ Lakehouse tables visible ({len(tables)}):")
    for table in tables['tableName'][:5]:  # Show first 5
        print(f"   📊 {table}")
    if len(tables) > 5:
        print(f"   ... and {len(tables)-5} more")
except Exception as e:
    print(f"❌ Cannot access lakehouse tables: {str(e)}")

# Check 4: File system access
try:
    files_path = "Files"
    if os.path.exists(files_path):
        print(f"✅ File system access: {files_path} exists")
    else:
        print(f"⚠️  File system: {files_path} not found")
except Exception as e:
    print(f"❌ File system access failed: {str(e)}")

# Check 5: SQL database access (will be checked after database name is defined)
print("⏳ SQL Database access check will run after database configuration...")

print("\n📋 Diagnostic complete. Address any ❌ issues before proceeding.")

In [ ]:
# Database connection configuration for Fabric environment
# Using Fabric's built-in Spark SQL connectivity
database_name = "Gaiye-SQL-DB"
schema_name = "SalesLT"

print(f"🔗 Connecting to Fabric SQL database: {database_name}")
print(f"📊 Target schema: {schema_name}")
print(f"🔐 Authentication: Fabric built-in connectivity")

# Test basic Spark SQL functionality
try:
    # Simple test query
    test_df = spark.sql("SELECT 1 as test_connection").toPandas()
    print("✅ Fabric Spark SQL connection successful")
    
    # Now test SQL database access
    try:
        test_query = f"SELECT COUNT(*) as table_count FROM {database_name}.information_schema.tables"
        result = spark.sql(test_query).toPandas()
        table_count = result.iloc[0]['table_count']
        print(f"✅ SQL Database access successful: {database_name} has {table_count} tables")
    except Exception as db_error:
        print(f"⚠️  SQL Database access issue: {str(db_error)}")
        print("💡 Trying alternative database access patterns...")
        
        # Try without database prefix
        try:
            alt_query = "SELECT COUNT(*) as table_count FROM information_schema.tables"
            result = spark.sql(alt_query).toPandas()
            table_count = result.iloc[0]['table_count']
            print(f"✅ Alternative database access successful: Found {table_count} tables")
        except Exception as alt_error:
            print(f"❌ Alternative database access failed: {str(alt_error)}")
            print("💡 Please ensure:")
            print(f"   1. {database_name} is linked to this Fabric workspace")
            print("   2. This notebook has a lakehouse attached")
            print("   3. You have proper permissions")
    
except Exception as e:
    print(f"❌ Basic Spark SQL test failed: {str(e)}")
    print("💡 Ensure you're running this notebook in Fabric with Spark enabled")

In [ ]:
# Discover all tables in SalesLT schema using Fabric SQL
print("🔍 Discovering tables in SalesLT schema using Fabric connectivity...")

try:
    # Method 1: Try using Fabric SQL Warehouse query
    table_discovery_query = f"""
    SELECT table_name as TABLE_NAME
    FROM {database_name}.information_schema.tables 
    WHERE table_schema = '{schema_name}' 
    AND table_type = 'BASE TABLE'
    ORDER BY table_name
    """
    
    print(f"? Executing query: {table_discovery_query}")
    
    # Execute using Spark SQL
    tables_spark_df = spark.sql(table_discovery_query)
    tables_df = tables_spark_df.toPandas()
    
    # Extract table names
    table_names = tables_df['TABLE_NAME'].tolist()
    
    print(f"✅ Found {len(table_names)} tables in {schema_name} schema:")
    for i, table in enumerate(table_names, 1):
        print(f"   {i}. {table}")
        
except Exception as e:
    print(f"❌ Fabric SQL query failed: {str(e)}")
    print("🔄 Trying alternative method...")
    
    try:
        # Method 2: Try direct database access pattern
        alternative_query = f"""
        SELECT table_name as TABLE_NAME
        FROM information_schema.tables 
        WHERE table_schema = '{schema_name}' 
        AND table_type = 'BASE TABLE'
        ORDER BY table_name
        """
        
        # Try without database prefix
        tables_spark_df = spark.sql(alternative_query)
        tables_df = tables_spark_df.toPandas()
        table_names = tables_df['TABLE_NAME'].tolist()
        
        print(f"✅ Alternative method successful - Found {len(table_names)} tables:")
        for i, table in enumerate(table_names, 1):
            print(f"   {i}. {table}")
            
    except Exception as e2:
        print(f"❌ Alternative method also failed: {str(e2)}")
        print("💡 Manual table list fallback...")
        
        # Fallback: Common SalesLT tables (manual list as last resort)
        table_names = [
            'Address',
            'Customer', 
            'CustomerAddress',
            'Product',
            'ProductCategory',
            'ProductDescription',
            'ProductModel',
            'ProductModelProductDescription',
            'SalesOrderDetail',
            'SalesOrderHeader'
        ]
        
        print(f"⚠️  Using fallback table list ({len(table_names)} common SalesLT tables):")
        for i, table in enumerate(table_names, 1):
            print(f"   {i}. {table}")
        print("💡 Note: This is a fallback list - actual schema may have different tables")

In [ ]:
# Define enhanced export function using Fabric Spark SQL
def export_table_to_bronze_fabric(table_name, database_name, schema_name="SalesLT"):
    """
    Export a single table to bronze layer using Fabric's Spark SQL connectivity
    """
    try:
        logger.info(f"Starting Fabric export for table: {table_name}")
        
        # Method 1: Try with database prefix
        try:
            query = f"SELECT * FROM {database_name}.{schema_name}.{table_name}"
            print(f"🔍 Trying query: {query}")
            
            # Execute using Spark SQL
            spark_df = spark.sql(query)
            df = spark_df.toPandas()
            
        except Exception as e1:
            print(f"⚠️  Database prefix failed, trying alternative: {str(e1)}")
            
            # Method 2: Try without database prefix
            try:
                query = f"SELECT * FROM {schema_name}.{table_name}"
                print(f"🔍 Trying alternative query: {query}")
                
                spark_df = spark.sql(query)
                df = spark_df.toPandas()
                
            except Exception as e2:
                print(f"⚠️  Schema prefix failed, trying table only: {str(e2)}")
                
                # Method 3: Try table name only (if schema is default)
                query = f"SELECT * FROM {table_name}"
                print(f"🔍 Trying table-only query: {query}")
                
                spark_df = spark.sql(query)
                df = spark_df.toPandas()
        
        # Handle data type conversions for Parquet compatibility
        for col in df.columns:
            if df[col].dtype == 'object':
                # Convert object columns to string to handle mixed types
                df[col] = df[col].astype(str)
            elif 'datetime' in str(df[col].dtype):
                # Ensure datetime columns are properly formatted
                df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Add comprehensive metadata
        df['_source_table'] = f"{schema_name}.{table_name}"
        df['_extraction_timestamp'] = datetime.now()
        df['_source_database'] = database_name
        df['_extraction_method'] = 'fabric_spark_sql'
        df['_record_count'] = len(df)
        
        # Create bronze layer directory structure
        bronze_path = f"Files/bronze/saleslt/{table_name.lower()}"
        os.makedirs(bronze_path, exist_ok=True)
        
        # Save to bronze layer as Parquet
        parquet_file = f"{bronze_path}/{table_name.lower()}.parquet"
        df.to_parquet(parquet_file, index=False)
        
        # Also save schema information
        schema_info = {
            'table_name': table_name,
            'column_count': len(df.columns),
            'columns': [{'name': col, 'dtype': str(df[col].dtype)} for col in df.columns],
            'extraction_timestamp': datetime.now().isoformat(),
            'query_used': query
        }
        
        with open(f"{bronze_path}/schema_info.json", 'w') as f:
            import json
            json.dump(schema_info, f, indent=2)
        
        logger.info(f"✅ Successfully exported {table_name}: {len(df)} rows")
        
        return {
            'table_name': table_name,
            'status': 'success',
            'row_count': len(df),
            'column_count': len(df.columns),
            'columns': list(df.columns),
            'bronze_path': bronze_path,
            'file_size_mb': round(df.memory_usage(deep=True).sum() / 1024 / 1024, 2),
            'parquet_file': parquet_file,
            'query_used': query
        }
        
    except Exception as e:
        error_msg = f"❌ Error exporting {table_name}: {str(e)}"
        logger.error(error_msg)
        
        return {
            'table_name': table_name,
            'status': 'failed',
            'error': str(e),
            'row_count': 0
        }

print("📝 Fabric-compatible export function defined")
print("🔧 Features: Spark SQL connectivity, multiple query patterns, fallback methods")

In [ ]:
# Execute export for all discovered tables using Fabric connectivity
print(f"🚀 Starting Fabric-based export of {len(table_names)} tables to bronze layer...\n")

export_results = []
successful_exports = 0
failed_exports = 0

for i, table_name in enumerate(table_names, 1):
    print(f"📊 Processing table {i}/{len(table_names)}: {table_name}")
    
    # Export table using Fabric method
    result = export_table_to_bronze_fabric(table_name, database_name, schema_name)
    export_results.append(result)
    
    # Track success/failure
    if result['status'] == 'success':
        successful_exports += 1
        print(f"   ✅ {result['row_count']} rows exported ({result.get('file_size_mb', 0)} MB)")
        if 'query_used' in result:
            print(f"   📝 Query: {result['query_used']}")
    else:
        failed_exports += 1
        print(f"   ❌ Export failed: {result.get('error', 'Unknown error')}")
    
    print()  # Empty line for readability

print(f"🎯 Fabric export completed!")
print(f"✅ Successful: {successful_exports} tables")
print(f"❌ Failed: {failed_exports} tables")

# If all exports failed, provide troubleshooting guidance
if failed_exports == len(table_names) and len(table_names) > 0:
    print("\n🚨 ALL EXPORTS FAILED - TROUBLESHOOTING GUIDE:")
    print("=" * 50)
    print("1. ✅ Verify Gaiye-SQL-DB is linked to this Fabric workspace")
    print("2. ✅ Ensure this notebook has a lakehouse attached")
    print("3. ✅ Check that you have read permissions on the SQL database")
    print("4. ✅ Confirm the SalesLT schema exists in Gaiye-SQL-DB")
    print("5. ✅ Try running: spark.sql('SHOW DATABASES').show() to see available databases")
    print("\n💡 Alternative: Use Data Factory pipeline instead of notebook for better connectivity")

In [ ]:
# Generate detailed export summary and pipeline configuration
print("📋 EXPORT SUMMARY REPORT")
print("=" * 50)
print(f"Export Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Source Database: {database_name}")
print(f"Source Schema: {schema_name}")
print(f"Destination: Bronze Layer (Files/bronze/saleslt/)")
print()

# Create summary DataFrame
summary_df = pd.DataFrame(export_results)

if not summary_df.empty:
    # Display successful exports
    successful_df = summary_df[summary_df['status'] == 'success']
    if not successful_df.empty:
        print("✅ SUCCESSFUL EXPORTS:")
        print(successful_df[['table_name', 'row_count', 'column_count', 'file_size_mb']].to_string(index=False))
        print()
        
        total_rows = successful_df['row_count'].sum()
        total_size = successful_df['file_size_mb'].sum()
        total_tables = len(successful_df)
        print(f"📊 Total tables exported: {total_tables}")
        print(f"📊 Total rows exported: {total_rows:,}")
        print(f"💾 Total data size: {total_size:.2f} MB")
        print()
    
    # Display failed exports
    failed_df = summary_df[summary_df['status'] == 'failed']
    if not failed_df.empty:
        print("❌ FAILED EXPORTS:")
        print(failed_df[['table_name', 'error']].to_string(index=False))
        print()

# Create comprehensive summary report
summary_report = {
    'pipeline_info': {
        'name': 'SalesLT Dynamic Export Pipeline',
        'type': 'schema_discovery_export',
        'authentication': 'fabric_spark_sql',
        'method': 'self_contained_notebook'
    },
    'export_metadata': {
        'export_timestamp': datetime.now().isoformat(),
        'source_database': database_name,
        'source_schema': schema_name,
        'destination_path': 'Files/bronze/saleslt/',
        'total_tables_discovered': len(table_names),
        'successful_exports': successful_exports,
        'failed_exports': failed_exports
    },
    'table_inventory': table_names,
    'export_details': export_results,
    'data_factory_integration': {
        'server_format': f'{database_name}.sql.fabric.microsoft.com',
        'authentication_type': 'organizational_account',
        'bronze_path_pattern': 'Files/bronze/saleslt/{table_name_lower}/{table_name_lower}.parquet',
        'metadata_pattern': 'Files/bronze/saleslt/{table_name_lower}/schema_info.json'
    }
}

# Save summary as JSON in bronze layer
summary_path = "Files/bronze/saleslt/_pipeline_summary.json"
os.makedirs("Files/bronze/saleslt", exist_ok=True)
with open(summary_path, 'w') as f:
    json.dump(summary_report, f, indent=2, default=str)

print(f"📄 Pipeline summary saved to: {summary_path}")
print("\n🎉 SalesLT dynamic schema export to bronze layer completed!")
print("🔗 Ready for Data Factory integration or silver layer processing")

In [ ]:
# Validation: Verify files exist in bronze layer
print("🔍 VALIDATION: Checking bronze layer contents...")
print("=" * 50)

bronze_base_path = "Files/bronze"

try:
    # List bronze layer directories
    if os.path.exists(bronze_base_path):
        bronze_contents = os.listdir(bronze_base_path)
        
        print(f"📁 Bronze layer contains {len(bronze_contents)} items:")
        for item in sorted(bronze_contents):
            item_path = os.path.join(bronze_base_path, item)
            if os.path.isdir(item_path):
                # Check for parquet files in the directory
                files = [f for f in os.listdir(item_path) if f.endswith('.parquet')]
                print(f"   📂 {item}/ ({len(files)} parquet files)")
            else:
                print(f"   📄 {item}")
    else:
        print(f"❌ Bronze layer path not found: {bronze_base_path}")
        
except Exception as e:
    print(f"❌ Error validating bronze layer: {str(e)}")

print("\n✅ Validation completed")

## Next Steps

After running this notebook, you should have:

1. **Exported Data**: All SalesLT tables saved as Parquet files in the bronze layer
2. **Metadata**: Each file includes source tracking and extraction timestamps
3. **Summary Report**: JSON file with complete export details
4. **File Organization**: Each table in its own folder within `/Files/bronze/`

### Recommended Next Actions:

- **Review the data**: Check the bronze layer files to ensure data quality
- **Create silver layer transformations**: Build data pipelines to clean and standardize the data
- **Map to retail model**: Align the SalesLT schema with your retail data model requirements
- **Set up monitoring**: Create alerts for data freshness and quality

### Troubleshooting:

- **Connection issues**: Verify your Fabric workspace has access to the SQL database
- **Permission errors**: Ensure you have read access to SalesLT schema and write access to the lakehouse
- **Large tables**: For very large tables, consider implementing chunked processing
- **Data types**: Some SQL Server data types may need special handling during export

## Data Factory Pipeline Alternative

If you prefer using **Data Factory** instead of this notebook, here's how to create a dynamic pipeline:

### 1. Create Parameterized Pipeline

Create a Data Factory pipeline with these components:

**Pipeline Parameters:**
- `sourceSchema` = "SalesLT"
- `targetPath` = "Files/bronze/saleslt"

**Activities:**
1. **Lookup Activity**: Get table list from `INFORMATION_SCHEMA.TABLES`
2. **ForEach Activity**: Loop through discovered tables  
3. **Copy Data Activity**: Copy each table dynamically

### 2. Connection Configuration

**Source (Fabric SQL Database):**
- **Server**: `Gaiye-SQL-DB.sql.fabric.microsoft.com`
- **Database**: `Gaiye-SQL-DB`
- **Authentication**: `Organizational Account`

**Sink (Lakehouse):**
- **Path**: `@concat(pipeline().parameters.targetPath, '/', toLower(item().TABLE_NAME), '/', toLower(item().TABLE_NAME), '.parquet')`
- **Format**: `Parquet`

### 3. Dynamic Query

**Lookup Query:**
```sql
SELECT TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_SCHEMA = '@{pipeline().parameters.sourceSchema}' 
AND TABLE_TYPE = 'BASE TABLE'
```

**Copy Data Source Query:**
```sql
SELECT * FROM [@{pipeline().parameters.sourceSchema}].[@{item().TABLE_NAME}]
```

This approach gives you the same dynamic table discovery but through Data Factory's visual interface.